In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [2]:
with open('url.json') as fopen:
    urls = sorted(json.load(fopen))

In [4]:
urls[:10]

['https://www.thevocket.com/1-5-juta-mat-bangla-sudah-tiba-masanya-ibubapa-ubah-mentaliti-anak-anak/',
 'https://www.thevocket.com/1-6-juta-penjawat-awam-terima-bonus-hari-ini/',
 'https://www.thevocket.com/1-bilion-akaun-yahoo-diceroboh/',
 'https://www.thevocket.com/1-utama-bakal-memperkenalkan-zon-peruncit-bertema-korea-district-k/',
 'https://www.thevocket.com/1-utama-dakwa-berlaku-salah-faham-dalam-laporan-44-pegawai-keselamatannya-positif-covid-19/',
 'https://www.thevocket.com/1-utama-jadi-pusat-membeli-belah-terbaru-catat-kes-positif-covid-19/',
 'https://www.thevocket.com/10-adab-ketika-melancong-di-luar-negara/',
 'https://www.thevocket.com/10-adegan-filem-ini-tidak-masuk-akal-tetapi-tetap-pecah-panggung/',
 'https://www.thevocket.com/10-ahli-parlimen-pkr-keluar-parti-bentuk-blok-bebas-parlimen-bersama-azmin-ali/',
 'https://www.thevocket.com/10-aktiviti-haram-di-korea-utara/']

In [5]:
r = requests.get(urls[0])
soup = BeautifulSoup(r.content, "lxml")

In [17]:
def crawl(url, filename):
    if os.path.exists(filename):
        return True
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    try:
        title = soup.find('h1', {'class': 'font-bold'}).text.strip()
    except:
        title = None
        
    try:
        p = soup.find('div', {'itemprop':"articleBody"}).find_all('p')
        p = [p_.text.strip() for p_ in p]
    except:
        p = None
    
    data = {
        'url': url,
        'title': title,
        'p': p,
    }
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)

In [18]:
!mkdir page

In [19]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

for i in tqdm(range(0, len(urls), max_worker)):
    aranged = np.arange(i, i + max_worker)
    urls_ = [(urls[a], os.path.join('page', f'{a}.json')) for a in aranged]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, url[0], url[1]): url for url in urls_}

    for future in as_completed(futures):
        future.result()

100%|██████████████████████████████████████▉| 2508/2509 [30:12<00:00,  1.38it/s]


IndexError: list index out of range